# #5. Black-Litterman model

#### #5.9 블랙-리터만 모델 최적화 (p 199)

Optimization condition for BL model:

\begin{align*}
    \max \Sigma_{i=1}^n \mu BL_i &- \frac{\lambda}{2} \Sigma_{i=1}^n \Sigma_{j=1}^n \sigma_{ij} w_i w_j \\
    \text{Subject to} \quad \Sigma_{i=1}^n w_i &=
    
     1, w_i \geq 0. i=1, 2, 3, \ldots, n
\end{align*}

; this enables us to compute the weights $w_i$ for asset allocation.

아래는 Sharpe 비율을 최대로하는 tangent portfolio 최적화를 수행하는 코드다:

In [1]:
import matplotlib.pylab as plt
import numpy as np
from numpy.linalg import inv
import pandas as pd
from pandas_datareader import data as web
from scipy.optimize import minimize

In [2]:
# 최적의 비중 계산을 위한 목적함수
# obj. funciton for optimizing weights
def solveWeights(R, C, rf):
    def obj(W, R, C, rf):
        mean = sum (R * W)
        var = np.dot(np.dot(W, C), W)
        
        # Sharpe's ratio as utility function: want to maximize
        util = (mean -rf) / np.sqrt(var)
        return 1 / util
    
    # number of assets
    n = len(R)
    
    # initial weights
    W = np.ones([n]) / n
    
    # doesn't allow short-selling nor borrowing
    # 비중 범위는 0-100% 사이: 공매도나 차입조건이 없음
    bnds = [(0., 1.) for i in range(n)]
    
    # constraints: sum of weights = 1
    cons = ({'type': 'eq', 'fun': lambda W: sum(W) - 1.})

    # W는 초기값, (R, C, rf)는 추가 인수
    res = minimize(obj, W, (R, C, rf), method='SLSQP', constraints=cons, bounds=bnds)

    # res.success: 최적화(minimize())가 성공적으로 수행되었는지를 나타내는 Boolean 값
    # False이면 res.message로 그 원인을 알 수 있다
    if not res.success:
        raise BaseException(res.message)
    
    return res.x

In [4]:
# example 1
R = np.array([0.05, 0.08])  # 자산 기대수익률
C = np.array([[0.01, 0.02],
              [0.02, 0.04]])  # 자산 공분산
rf = 0.02  # 무위험 이자율
weights = solveWeights(R, C, rf)  # 최적 비중 계산

print("Optimal Weights:", weights)

Optimal Weights: [0.5 0.5]


In [5]:
# example 2
R = [0.05, 0.08, 0.12]  # 자산의 기대수익률
C = [[0.04, 0.02, 0.01],
     [0.02, 0.09, 0.03],
     [0.01, 0.03, 0.06]]  # 자산들 간의 공분산 행렬
rf = 0.03  # 무위험 이자율

weights = solveWeights(R, C, rf)
print(weights)

[0.07040263 0.02735284 0.90224453]


In [9]:
# 무위험 수익률, 수익률, Cov로 효율적 투자선 계산
# calculating Efficient Frontier via risk-less rate, return rate and the Covariance
def solveFrontier(R, C, rf):

    # 최적 비중 계산을 위한 목적함수
    def obj(W, R, C, r):

        # 주어진 수익률에서 분산을 최소화하는 비중 계산
        mean = sum(R * W)
        var = np.dot(np.dot(W, C), W)

        # 최적화 제약조건 페널티
        penalty = 100 * abs(mean - r)
        return var + penalty
    
    # eff. frontier의 평균과 분산이 될 list
    frontier_mean, frontier_var = [ ], [ ]
    n = len(R)
    
    # 수익률 최저에서 최대 사이에서 목표 수익률(r)을 다양하게 설정하여 포트폴리오 최적화를 반복
    for r in np.linspace(min(R), max(R), num=20):
        W = np.ones([n]) / n
        bnds = [(0, 1) for i in range(n)]
        cons = [{'type': 'eq', 'fun': lambda W: sum(W) - 1.}]

        res = minimize(obj, W, (R, C, r), method='SLSQP', constraints=cons, bounds=bnds)

        if not res.success:
            raise BaseException(res.message)
        
        frontier_mean.append(r)
        frontier_var.append(np.dot(np.dot(res.x, C), res.x))

    return np.array(frontier_mean), np.array(frontier_var)

위 코드에서 obj. f'tn의 penalty 항을 주목해보자:

이러한 패널티는 효율적인 포트폴리오 최적화를 위해 사용된다. 목표 수익률과 실제 포트폴리오의 수익률이 일치하는 경우에는 패널티가 없다. 그러나 목표 수익률과 포트폴리오 수익률 간에 차이가 클수록 패널티가 증가한다.

이를 통해, 목표 수익률과 가능한 작은 분산을 동시에 달성하는 포트폴리오를 찾기 위해 최적화 과정에서 목표 수익률에 가까운 포트폴리오를 선호하도록 유도한다. 따라서 목적함수에 패널티 항을 추가하여 수익률과 분산의 트레이드오프를 조정하고 최적의 포트폴리오를 찾을 수 있게 된다.

In [10]:
# example
R = np.array([0.08, 0.12, 0.15])  # 수익률
C = np.array([[0.02, 0.005, 0.01], 
              [0.005, 0.03, 0.02], 
              [0.01, 0.02, 0.04]])  # 공분산
rf = 0.05  # 무위험 이자율

# Efficient Frontier 계산
frontier_mean, frontier_var = solveFrontier(R, C, rf)

# 결과 출력
for mean, var in zip(frontier_mean, frontier_var):
    print(f"Mean Return: {mean:.4f}, Variance: {var:.4f}")

Mean Return: 0.0800, Variance: 0.0200
Mean Return: 0.0837, Variance: 0.0176
Mean Return: 0.0874, Variance: 0.0158
Mean Return: 0.0911, Variance: 0.0148
Mean Return: 0.0947, Variance: 0.0144
Mean Return: 0.0984, Variance: 0.0152
Mean Return: 0.1021, Variance: 0.0146
Mean Return: 0.1058, Variance: 0.0151
Mean Return: 0.1095, Variance: 0.0158
Mean Return: 0.1132, Variance: 0.0167
Mean Return: 0.1168, Variance: 0.0178
Mean Return: 0.1205, Variance: 0.0192
Mean Return: 0.1242, Variance: 0.0208
Mean Return: 0.1279, Variance: 0.0225
Mean Return: 0.1316, Variance: 0.0246
Mean Return: 0.1353, Variance: 0.0268
Mean Return: 0.1389, Variance: 0.0292
Mean Return: 0.1426, Variance: 0.0320
Mean Return: 0.1463, Variance: 0.0355
Mean Return: 0.1500, Variance: 0.0400


In [ ]:
# 효율적 포트폴리오 최적화
def optimize_frontier(R, C, rf):

    # tangency portfolio 계산
    W = solveWeights(R, C, rf)

    tan_mean = sum(R * W)
    tan_var = np.dot(np.dot(W, C), W)

    # 효율적 포트폴리오 계산
    eff_mean, eff_var = solveFrontier(R, C, rf)
    
    # dict 타입으로 리턴
    return {'weights':W, 'tan_mean':tan_mean, 'tan_var':tan_var, 'eff_mean':eff_mean, 'eff_var':eff_var}

In [ ]:
# 자산에 대한 투자자의 전망 (행렬)과 전망의 기대수익률 행렬
def CreateMatrixPQ(names, views):
    r, c = len(views), len(names)

    # views[i][3] = exp. return(기대 수익률)
    Q = [views[i][3] for i in range(r)]
    
    nameToIndex = dict()
    for i, n in enumerate(names):
        nameToIndex[n] = i
    
    P = np.zeros([r, c])
    for i, v in enumerate(views):
        name1, name2 = views[i][0], views[i][2]
        P[i, nameToIndex[name1]] = +1 if views[i][1] == '>' else -1
        P[i, nameToIndex[name2]] = -1 if views[i][1] == '>' else +1
    
    return np.array(Q), P

In [ ]:
# read the data
tickers = ['PFE','INTC','NFLX','JPM','XOM','GOOG','JNJ','AAPL','AMZN']
cap = {'PFE':201102000000,'INTC':257259000000,'NFLX':184922000000,
       'JPM':272178000000,'XOM':178228000000,'GOOG':866683000000,
       'JNJ':403335000000,'AAPL':1208000000000,'AMZN':1178000000000
      }

prices, caps = [ ], [ ]

for s in tickers:
    pxclose = web.DataReader(s, data_source='yahoo', start='01-01-2018', end='31-12-2019')['Adj Close']
    prices.append(list(pxclose))
    caps.append(cap[s])

In [ ]:
n = len(tickers)
W = np.array(caps) / sum(caps)
prices = np.matrix(prices)

rows, cols = prices.shape
returns = np.empty([rows, cols - 1])
for r in range(rows):
    for c in range(cols - 1):
        p0, p1 = prices[r, c], prices[r, c + 1]
        returns[r, c] = (p1 / p0) - 1

expreturns = np.array([])
for r in range(rows):
    expreturns = np.append(expreturns, np.mean(returns[r]))

covars = np.cov(returns)
R = (1 + expreturns) ** 250 - 1
C = covars * 250

rf = .015

In [ ]:
expreturns